# This notebook is used to automate the deployment of Trinity Framework ingestion pipelines in Fabric workspace
----------------------------------------------------------------------------------------------------------------
- **TrinityIngestionDataPipelineDeployment**
- **Version V1.0**

- **PREREQUISITES**
- Please ensure that the **JSON files & YAML files** are kept at the lakehouse folder.
----------------------------------------------------------------------------------------------------------------


# Install jsonpath_ng library. jsonpath_ng library provides functionality for querying JSON-like data structures using JSONPath expressions. 

In [1]:
%pip install jsonpath_ng

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 7, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



# Import statements for necessary libraries:

- `requests`: Used for making HTTP requests.
- `json`: Provides functions for encoding and decoding JSON data.
- `base64`: Used for encoding and decoding data in Base64 format.
- `sempy` library and its `fabric` module for working with Fabric in Sempy.
- `pyyaml` library is used for YAML parsing and emitter for Python.

In [2]:
# import libraries
import requests
import json
import base64
import sempy
import sempy.fabric as fabric
import yaml

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 9, Finished, Available)

# Get current workspace & session token

In [3]:
workspace_id = fabric.get_workspace_id()
token_string = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com/")
header = {'Content-Type':'application/json','Authorization': f'Bearer {token_string}'}
response = requests.get(url='https://api.fabric.microsoft.com/v1/workspaces', headers=header)

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 10, Finished, Available)

# Read and parse the configuration file

In [4]:
import yaml

base_folder_path = "/lakehouse/default/Files/"  # modify based on your environment e.g. Files/ in Fabric
config_file_name = "pipeline_deployment_config.yml"
config_file_path = base_folder_path + config_file_name
config_data = None
externalreference_id_map = {}  # Create a map to store oldconnectionid and newconnectionid

# Get details of data warehouse name, id & endpoint from configuration file
dest_dw_name = ""
dest_dw_object_id = ""
dest_dw_endpoint = ""

# Get details of lakehouse name, id & endpoint from configuration file
dest_lh_name = ""
dest_lh_object_id = ""
dest_lh_endpoint = ""

# Read the config file
try:
    with open(config_file_path, 'r') as file:
        config_data = yaml.safe_load(file)
        # Debug: Print the type and content of config_data
        print(f"Loaded config data (type: {type(config_data)}): {config_data}")
except yaml.YAMLError as exc:
    print(f"Error reading YAML file: {exc}")
except FileNotFoundError as fnf_error:
    print(f"Config file not found: {fnf_error}")

# Process the config data if available
if isinstance(config_data, dict):
    # Process pipeline configuration files
    pipeline_config_files = config_data.get("pipelinejsonfiles", [])

    # Read original pipeline reference IDs
    original_pipeline_ids = {}
    for item in config_data.get('originalpipelinereferenceids', []):
        display_name = item.get('pipelinedisplayname')
        pipeline_id = item.get('originalpipelineid')
        original_pipeline_ids[display_name] = pipeline_id

    # Process Destination Datawarehouse details
    datawarehouse_details = config_data.get("DestinationDatawarehousedetails", [])
    for item in datawarehouse_details:
        dest_dw_name = item.get("name")
        print(dest_dw_name)
        dest_dw_object_id = item.get("objectId")
        print(dest_dw_object_id)
        dest_dw_endpoint = item.get("endpoint")
        print(dest_dw_endpoint)

    # Process Destination Lakehouse details
    lakehouse_details = config_data.get("DestinationLakehousedetails", [])
    for item in lakehouse_details:
        dest_lh_name = item.get("name")
        print(dest_lh_name)
        dest_lh_object_id = item.get("objectId")
        print(dest_lh_object_id)
        dest_lh_endpoint = item.get("endpoint")
        print(dest_lh_endpoint)
    for pipeline in pipeline_config_files:
        external_refs = pipeline.get('externalreferences')
        if external_refs:  # Check if external references exist
                
                for ref in external_refs:
                    old_id = ref.get('oldconnectionid')
                    new_id = ref.get('newconnectionid')
                    if old_id and new_id:  # Ensure both IDs are present
                        externalreference_id_map[old_id] = new_id
                #print("Pipeline:", pipeline['pipelinedisplayname'])
                #print("ID Map:", id_map)
                #print()  # Add a newline for readability
else:
    print("No valid configuration data found.")

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 11, Finished, Available)

Loaded config data (type: <class 'dict'>): {'pipelinejsonfiles': [{'pipelinedisplayname': 'PL_Auditing_PG', 'pipelinejsonfilepath': '/lakehouse/default/Files/PL_Auditing_PG.json', 'invokepipeline': 'false', 'externalreferencesflg': 'false', 'internalreferencesflg': 'true'}, {'pipelinedisplayname': 'PL_PopulateMetadataTable_PG', 'pipelinejsonfilepath': '/lakehouse/default/Files/PL_PopulateMetadataTable_PG.json', 'invokepipeline': 'false', 'externalreferencesflg': 'true', 'internalreferencesflg': 'true', 'externalreferences': [{'type': 'PostgreSQL', 'oldconnectionid': 'f39ca659-7698-41f7-93c2-6ee3a115febc', 'newconnectionid': '083d109b-2c2a-4fef-8581-69d9e1903649'}]}, {'pipelinedisplayname': 'PL_SendEmailNotification_PG', 'pipelinejsonfilepath': '/lakehouse/default/Files/PL_SendEmailNotification_PG.json', 'invokepipeline': 'false', 'externalreferencesflg': 'false', 'internalreferencesflg': 'false'}, {'pipelinedisplayname': 'PL_DynamicIngestionPipelineIncrmLoad_PG', 'pipelinejsonfilepath'

# Function to feed a string and it will output a base64 string.

In [5]:
def b64EncodeString(json_str):
    # Encode the string using Base64
    base64_encoded = base64.b64encode(json_str.encode('utf-8'))
    base64_encoded_str = base64_encoded.decode('utf-8')
    return base64_encoded_str

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 12, Finished, Available)

# Read JSON file from Lakehouse location

In [6]:
def readJsonFile(file_path):
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            # Remove 'name' and 'objectId' properties
            if 'name' in json_data:
                del json_data['name']
            if 'objectId' in json_data:
                del json_data['objectId']
            return json_data
            #print(json_data)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file '{file_path}'.")

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 13, Finished, Available)

# Read & Replace Embedded PipelineIDs Json

In [7]:
def readandReplaceEmbeddedPipelineIDsJson(filepath,invoked_pipeline_list,successfully_created_pipelines):
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            for invoked_pipeline in invoked_pipeline_list:
                new_pipeline_id = successfully_created_pipelines.get(invoked_pipeline)
                original_pipeline_id = original_pipeline_ids.get(invoked_pipeline)
                json_data = replace_property(json_data,original_pipeline_id,new_pipeline_id)
            return json_data
            #print(json_data)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file '{file_path}'.")

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 14, Finished, Available)

# Replace individual property. This is a sub function used by readandReplaceEmbeddedPipelineIDsJson

In [8]:
def replace_property(json_obj, old_value, new_value):
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if isinstance(value, (dict, list)):
                replace_property(value, old_value, new_value)
            elif value == old_value:
                json_obj[key] = new_value
    elif isinstance(json_obj, list):
        for item in json_obj:
            replace_property(item, old_value, new_value)
    return json_obj

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 15, Finished, Available)

# Replace external references connection value

In [9]:
from jsonpath_ng import parse  # Assuming you're using jsonpath-ng library

def replace_externalreferences_connection_value(json_data, old_value, new_value):
    
    # Define JSONPath expression to find the connection value
     # Define the JSONPath expression to find the connection value
    jsonpath_expr = parse('$..externalReferences.connection')
    # Find the matching node
    matches = jsonpath_expr.find(json_data)
    for match in matches:
        # The match's path is used to update the value
        jsonpath_expr.update(json_data, new_value)
    
    # Convert the updated data back to a JSON string
    updated_json = json_data
    
    return updated_json

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 16, Finished, Available)

# Replace linked service details

In [10]:
from jsonpath_ng import parse 
def replace_linked_service_details(json_data):
    jsonpath_expr = parse('$..linkedService')
    matches = jsonpath_expr.find(json_data)
    
    if matches:
        for match in matches:
            linked_service = match.value
            if(linked_service['properties']['type'] == 'DataWarehouse'):
                linked_service['name'] = dest_dw_name
                linked_service['objectId'] = dest_dw_object_id
                linked_service['properties']['typeProperties']['artifactId'] = dest_dw_object_id
                linked_service['properties']['typeProperties']['endpoint'] = dest_dw_endpoint
            elif(linked_service['properties']['type'] == 'Lakehouse'):
                linked_service['name'] = dest_lh_name
                linked_service['properties']['typeProperties']['artifactId'] = dest_lh_object_id
                linked_service['properties']['typeProperties']['workspaceId'] = workspace_id
    else:
        print(f"No matches found")
    return json_data

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 17, Finished, Available)

# This block is used to traverse through the list of pipelines that needs to deployed.

In [11]:
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items"
successfully_created_pipelines = dict()

for item in pipeline_config_files:
  display_name = item.get("pipelinedisplayname")
  file_path = item.get("pipelinejsonfilepath")
  invoke_pipeline = item.get("invokepipeline")
  externalreferencesflg = item.get("externalreferencesflg")
  internalreferencesflg = item.get("internalreferencesflg")
  invoked_pipeline_list = item.get("invokedpipelinelist", []) if invoke_pipeline == "true" else None
  
  if invoke_pipeline == "false":
    json_file_value =  readJsonFile(file_path)
    #print(json_file_value)
  elif invoke_pipeline == "true":
    json_file_value =  readandReplaceEmbeddedPipelineIDsJson(file_path,invoked_pipeline_list,successfully_created_pipelines)

  if internalreferencesflg == "true":
    json_file_value = replace_linked_service_details(json_file_value)

  if externalreferencesflg == "true":
    for key, value in externalreference_id_map.items():
      json_file_value = replace_externalreferences_connection_value (json_file_value,key,value)

  # Convert JSON object to string
  json_str = json.dumps(json_file_value) 

  base64_encoded_str = b64EncodeString(json_str)

  payload = json.dumps({
    "displayName": display_name,
    "type": "DataPipeline",
    "definition": {
      "parts": [
        {
          "path": "pipeline-content.json",
          "payload": base64_encoded_str,
          "payloadType": "InlineBase64"
        }
      ]
    }
  })
  headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token_string}'
  }
  response = requests.request("POST", url, headers=headers, data=payload)
  print(response.text)
  response_dict = json.loads(response.text)

  if response.ok:
    print(f">> Pipeline '{display_name}' deployed.")
  else:
    raise RuntimeError(f"Pipeline '{display_name}' creation failed: {response.status_code}: {response.text}")

  if "id" in response_dict:
    successfully_created_pipelines[response_dict['displayName']]= response_dict['id']

StatementMeta(, cd65226c-5e7f-4fc6-b230-80ce81de4489, 18, Finished, Available)

{"properties": {"activities": [{"name": "PL_Auditing", "description": "Auditing component is used to audit the records ingested into data lake. Auditing includes identifying any errors or issues that may have occurred during the process, as well as reviewing performance metrics to identify areas for improvement.", "type": "SqlServerStoredProcedure", "dependsOn": [], "policy": {"timeout": "0.12:00:00", "retry": 0, "retryIntervalInSeconds": 30, "secureOutput": false, "secureInput": false}, "typeProperties": {"storedProcedureName": "[mtd].[capture_audit_event_sp]", "storedProcedureParameters": {"copy_duration": {"value": {"value": "@pipeline().parameters.par_copy_duration", "type": "Expression"}, "type": "Int32"}, "data_consistency_verification": {"value": {"value": "@pipeline().parameters.par_data_consistency_verification", "type": "Expression"}, "type": "String"}, "data_read": {"value": {"value": "@pipeline().parameters.par_data_read", "type": "Expression"}, "type": "Int64"}, "data_writ